
Koronavirüs hastalığı 2019 (COVID-19), "şiddetli akut solunum sendromu koronavirüs 2" (SARS-CoV-2) virüs suşunun neden olduğu bulaşıcı bir hastalıktır. Hastalık ilk olarak 2019 yılında Çin'in Wuhan şehrinde tanımlandı ve o zamandan beri küresel olarak yayıldı ve 2019-20 koronavirüs pandemisine neden oldu.

Enfeksiyon genellikle öksürme ve hapşırma sırasında üretilen solunum damlacıkları yoluyla bir kişiden diğerine yayılır. Semptomların ortaya çıkmasına kadar geçen süre genellikle ortalama 5 gün olmak üzere 2 ila 14 gün arasındadır.

Dünya Sağlık Örgütü (WHO) 2019-20 koronavirüs salgını salgını ve Uluslararası Endişe Halk Sağlığı Acil Durumunu (PHEIC) ilan etti.

# Kütüphaneler

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Yüklenen Veriler

Johns Hopkins Üniversitesi, etkilenen vaka verilerini kullanarak mükemmel bir gösterge tablosu oluşturdu. Veriler ilişkili google sayfalarından çıkarılır.
Veriler 22 Ocak 2020'den itibaren mevcuttur.
### Kolon AÇIKLAMASI
  - Sno - Seri numarası
  - ObservationDate - AA / GG / YYYY cinsinden gözlem tarihi
  - İl / Eyalet - Gözlemin eyaleti veya eyaleti (Eksik olduğunda boş olabilir)
  - Ülke / Bölge - Gözlem ülkesi
  - Son Güncelleme - Satırın belirtilen il veya ülke için güncellendiği UTC saati. (Standart değil ve lütfen kullanmadan önce temizleyin)
  - Onaylandı - O tarihe kadar toplam onaylanmış vaka sayısı
  - Ölümler - O tarihe kadar biriken ölüm sayısı
  - Kurtarıldı - O tarihe kadar birikmiş geri kazanılmış vaka sayısı

# Veri Okuma

In [ ]:
covid19=pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
# covid19 değişkenine covid_19_data.csv dataframe i atadım.
# Tarihleri datetime'a dönüştürme
covid19['ObservationDate']=pd.to_datetime(covid19['ObservationDate'])

Veri goruntuleme
Görselleştirme, korona virüsünün bir ülkeden diğerine ve ne kadar yayıldığını anlamaya yardımcı olur.

In [ ]:
covid19.head()

In [ ]:
covid19.info()

# Veri Seti Tanıtımı

In [ ]:
covid19.describe()

In [ ]:
df=covid19.groupby(['ObservationDate'])[['Confirmed','Recovered','Deaths']].sum()
plt.figure(figsize=(20,10))
plt.title('22 Ocak 2020 den itibaren Novel Durumları',fontsize=30)
plt.xlabel('Date',fontsize=20)
plt.ylabel('Number of cases',fontsize=20)
plt.plot(df.index,df['Confirmed'],label='Enfekte',linewidth=3)
plt.plot(df.index,df['Recovered'],label='İyileşen',linewidth=3,color='green')
plt.plot(df.index,df['Deaths'],label='Ölen',linewidth=3,color='red')
plt.bar(df.index,df['Confirmed'],alpha=0.2,color='c')
plt.xticks(fontsize=15,rotation=90)
plt.yticks(fontsize=15)
plt.style.use('ggplot')
plt.legend()

In [ ]:
df1=covid19.groupby(['Country/Region'])[['ObservationDate','Confirmed','Recovered','Deaths']]
india_cases=df1.get_group('India')
plt.figure(figsize=(20,8))
plt.title('Hindistandaki Vaka Sayılari',fontsize=20)

plt.ylabel('Vaka Sayıları ',fontsize=20)
plt.xlabel('Tarihler',fontsize=20)
plt.plot(india_cases['ObservationDate'],india_cases['Confirmed'],'-o',linewidth=2,markersize=10,mfc='red',mew=2.9,mec='black')
plt.xticks(rotation=90)
plt.grid()

Hindistan'da 2019-20 koronavirüs pandemisinin ilk vakası 30 Ocak 2020'de Çin menşeli olarak rapor edildi. 3 Mart 2020'den sonra, enfekte vakalar Hindistan'da giderek artmaktadır. 20 Mart 2020 tarihi itibariyle, Sağlık ve Aile Refahı Bakanlığı ülkedeki toplam 223 vakayı ve 4 ölümü doğrulamıştır.

In [ ]:
country=covid19.groupby(['Country/Region'])[['Confirmed','Recovered','Deaths']].sum()
top_5=country.nlargest(5,['Confirmed'])
plt.figure(figsize=(20,16))
plt.subplot(311)
plt.title('Doğrulanmış, iyileşmiş ve ölüm vakası bulunan ilk 5 ülke',fontsize=20)
plt.barh(top_5.index,top_5['Confirmed'],color='blue')
plt.yticks(fontsize=20)
plt.xlabel('Doğrulanmış',fontsize=20)
plt.subplot(312)
plt.barh(top_5.index,top_5['Deaths'],color='red')
plt.yticks(fontsize=20)
plt.xlabel('Ölümler',fontsize=20)
plt.subplot(313)
plt.barh(top_5.index,top_5['Recovered'],color='green')
plt.yticks(fontsize=20)
plt.xlabel('İyileşen',fontsize=20)

In [ ]:
covid19['day']=covid19['ObservationDate'].dt.day
import matplotlib.ticker as ticker
cv1=covid19[covid19['ObservationDate']>'2020-03']
fig, ax=plt.subplots(figsize=(15,8))
def draw_barchart(date):
    df=cv1[cv1['day'].eq(date)].sort_values(by='Confirmed',ascending=True).tail(10)
    ax.clear()
    ax.text(0,1.12,'18 Mart 2020 de en çok enfekte olan ülkeler',size=24,weight=600,transform=ax.transAxes,ha='left')
    ax.barh(df['Country/Region'],df['Confirmed'],color='orange')
    for i, (country,value) in enumerate(zip(df['Country/Region'],df['Confirmed'])):
        ax.text(value,i, country, size=14, ha='right',va='bottom')
        ax.text(value,i,f'{value:.0f}', size=14, ha='left',va='center')
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}')) 
    ax.set_yticks([])
    ax.set_axisbelow(True)
    ax.margins(0,0.1)
    ax.tick_params(axis='x',labelsize=15,colors='blue')
    ax.grid(which='major',axis='x',linestyle='--')
    plt.box(False) 
draw_barchart(18)


In [ ]:
rank=country.nlargest(179,['Confirmed']).head(10)
confirmed=[]
recovered=[]
death=[]
for i in rank.index:
    df1=covid19[covid19['Country/Region']==i]
    confirmed.append(df1['Confirmed'].mean())
    recovered.append(df1['Recovered'].mean())
    death.append(df1['Deaths'].mean())
plt.figure(figsize=(20,20))

plt.subplot(311)
plt.title('Ortalamaları doğrulanmış, iyileşmiş ve ölüm vakaları olan ilk 10 ülke',fontsize=20,color='green')
plt.plot(rank.index,confirmed,'-o',mfc='black')
plt.ylabel('Onaylanmış',fontsize=20)
plt.grid()
plt.subplot(312)
plt.plot(rank.index,recovered,'-o',color='green',mfc='black')
plt.ylabel('İyileşen',fontsize=20)
plt.grid()
plt.subplot(313)
plt.plot(rank.index,death,'-o',color='red',mfc='black')
plt.ylabel('Ölümler',fontsize=20)
plt.grid()    

In [ ]:
rank1=country.nlargest(179,['Confirmed']).head(20)
confirmed_perc=[]
for i in rank1['Confirmed']:
    confirmed_perc.append(i/rank1['Confirmed'].sum())
plt.figure(figsize=(20,20))    
plt.title('Onaylanan Vakaların Dağılımı',fontsize=20)    
plt.pie(confirmed_perc,autopct='%1.1f%%')
plt.legend(rank1.index,loc='best')
plt.show()    

## Belirtiler

Virüs ile enfekte olanlar asemptomatik olabilse de, çoğu ateş, öksürük ve nefes darlığı gibi grip benzeri semptomlar geliştirir.

In [ ]:
symptoms={'Belirtiler':['Ateş',
        'Kuru Öksürük',
        'Yorgunluk',
        'Balgam',
        'Nefes Darlığı',
        'Kas Ağrısı',
        'Boğaz Ağrısı',
        'Bağ Ağrısı',
        'Titreme',
        'Mide Bulantısı',
        'Burun Tıkanıklığı',
        'İshal',
        'Hemoptizi',
        'Konjonktival tıkanıklık'],'Yüzdelik Oran':[87.9,67.7,38.1,33.4,18.6,14.8,13.9,13.6,11.4,5.0,4.8,3.7,0.9,0.8]}

symptoms=pd.DataFrame(data=symptoms,index=range(14))
symptoms

In [ ]:
plt.figure(figsize=(20,15))
plt.title('Belirtilerin Dağılımı',fontsize=20)    
plt.pie(symptoms['Yüzdelik Oran'],autopct='%1.1f%%')
plt.legend(symptoms['Belirtiler'],loc='best')
plt.show() 

## Önleme
Kendinizi koruyabilir ve aşağıdaki durumlarda virüsün başkalarına yayılmasını önlemeye yardımcı olabilirsiniz:

Yapılması gerekenler
  - Ellerinizi düzenli olarak 20 saniye boyunca sabun ve su veya alkol bazlı el ovma ile yıkayın.
  - Öksürdüğünüzde veya hapşırdığınızda burnunuzu ve ağzınızı tek kullanımlık bir doku veya bükülmüş dirsekle örtün
  - Hasta olan kişilerle yakın temastan (1 metre veya 3 feet) kaçının
  - Kendinizi iyi hissetmiyorsanız, evde kalın ve evdeki diğer insanlardan ayrı tutun
  
Yapılmaması gerekenler

Elleriniz temiz değilse gözlerinize, burnunuza veya ağzınıza dokunun

## Bina Modeli

Virüsün farklı ülkelere ve bölgelere nasıl yayılabileceğini tahmin eden güçlü bir model oluşturmalıyız. Bu görevin amacı, Mart 2020 boyunca virüsün ilerlemesini öngören bir model oluşturmaktır.

In [ ]:
covid19['Country/Region']=covid19['Country/Region'].astype('str')
covid19['Province/State']=covid19['Province/State'].astype('str')
covid19['day']=covid19['ObservationDate'].dt.day
covid19['month']=covid19['ObservationDate'].dt.month
lbl=preprocessing.LabelEncoder()
for c in ['Province/State','Country/Region']:
    lbl.fit(covid19[c].unique())
    covid19[c]=lbl.transform(covid19[c])
x=covid19.drop(['Confirmed','SNo','Last Update','ObservationDate'],1)
y=covid19['Confirmed']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
model=LinearRegression()
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)
from sklearn.metrics import r2_score

print(' R2 Score   : ',r2_score(y_test, y_pred))

In [ ]:
y_pred = model.predict(x_test)

df_predictions = pd.DataFrame()
df_predictions['y_pred_log'] = y_pred
df_predictions['y_true_log'] = y_test.values
df_predictions['y_pred'] = 10 ** y_pred
df_predictions['y_true'] = 10 ** y_test.values

df_predictions['absolute_pct_error'] = abs((df_predictions['y_pred'] - df_predictions['y_true']) / df_predictions['y_true']) * 100